# Copyright Netherlands eScience Center <br>
** Function     : Sample trained BayesConvLSTM and save the weights to ConvLSTM** <br>
** Author       : Yang Liu ** <br>
** First Built  : 2020.06.09 ** <br>
** Last Update  : 2020.11.09 ** <br>
** Library      : Pytorth, Numpy, NetCDF4, os, iris, cartopy, dlacs, matplotlib **<br>
Description     : This notebook serves to test the fixed sampling of BayesConvLSTM and the initialization of ConvLSTM weight. This could help to test the sampling weight of a trained BayesConvLSTM neural network. <br>

Return Values   : pkl model <br>

In [1]:
%matplotlib inline

import sys
import warnings
import numbers

# for data loading
import os
from netCDF4 import Dataset
# for pre-processing and machine learning
import numpy as np
#import sklearn
#import scipy
import torch
import torch.nn.functional

#sys.path.append(os.path.join('C:','Users','nosta','ML4Climate','Scripts','DLACs'))
sys.path.append("C:\\Users\\nosta\\ML4Climate\\Scripts\\DLACs")
import dlacs
import dlacs.BayesConvLSTM
import dlacs.ConvLSTM
import dlacs.preprocess
import dlacs.function
import dlacs.saveNetCDF

# for visualization
import dlacs.visual
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import iris # also helps with regriding
import cartopy
import cartopy.crs as ccrs

# ignore all the DeprecationWarnings by pytorch
if not sys.warnoptions:
    warnings.simplefilter("ignore")

The testing device is Dell Inspirion 5680 with Intel Core i7-8700 x64 CPU and Nvidia GTX 1060 6GB GPU.<br>
Here is a benchmark about cpu v.s. gtx 1060 <br>
https://www.analyticsindiamag.com/deep-learning-tensorflow-benchmark-intel-i5-4210u-vs-geforce-nvidia-1060-6gb/

In [2]:
################################################################################# 
#########                           modelpath                            ########
#################################################################################
# path of initialization weight
#bayesconvlstm_model_path = 'C:\\Users\\nosta\\ML4Climate\\PredictArctic\\BayesMaps\\Lorenz84\\bayes\\fullseries\\penalty100'
bayesconvlstm_model_path = 'C:\\Users\\nosta\\ML4Climate\\PredictArctic\\BayesMaps\\Lorenz84\\bayes\\fullseries_ex\\epoch3000_p100000_stdx4'
# path of output
output_path = 'C:\\Users\\nosta\\ML4Climate\\PredictArctic\\BayesMaps'

In [3]:
if __name__=="__main__":
    print ('*******************  create basic dimensions for network  *********************')
    # specifications of neural network
    input_channels = 3
    #hidden_channels = [3, 2, 1] # number of channels & hidden layers, the channels of last layer is the channels of output, too
    #hidden_channels = [3, 3, 3, 3, 2]
    hidden_channels = [3]
    kernel_size = 1
    batch_size = 1
    #num_layers = 1
    learning_rate = 0.01
    num_epochs = 3000

*******************  create basic dimensions for network  *********************


In [4]:
    print ('*******************  check the environment  *********************')
    print ("Pytorch version {}".format(torch.__version__))
    # check if CUDA is available
    use_cuda = torch.cuda.is_available()
    print("Is CUDA available? {}".format(use_cuda))
    # CUDA settings torch.__version__ must > 0.4
    # !!! This is important for the model!!! The first option is gpu
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

*******************  check the environment  *********************
Pytorch version 1.1.0
Is CUDA available? True


In [5]:
    %%time
    print ('*******************  load exsited LSTM model  *********************')
    # load model parameters
    model_bayes = dlacs.BayesConvLSTM.BayesConvLSTM(input_channels, hidden_channels, kernel_size).to(device)
    model_bayes.load_state_dict(torch.load(os.path.join(bayesconvlstm_model_path,
                                'bayesconvlstm_lorenz84_ex_epoch3000_p100000_stdx4.pkl'), map_location=device))
#                                'bayesconvlstm_lorenz84_fullseries_penalty100.pkl'), map_location=device))
    print(model_bayes)

*******************  load exsited LSTM model  *********************
!@#$% The network will be built with reduced size BayesConvLSTM cell. !@#$%
BayesConvLSTM(
  (cell0): BayesConvLSTMCell()
)
Wall time: 7.53 s


In [6]:
    print ('*******************  check wieght matrix of BayesConvLSTM  *********************')
    for name, param in model_bayes.named_parameters():
        if param.requires_grad:
            print (name)
            print (param.data)
            print (param.size())
            print ("=========================")

*******************  check wieght matrix of BayesConvLSTM  *********************
cell0.Wxi_mu
tensor([[[[ 1.8897]],

         [[ 0.9244]],

         [[-1.5973]]],


        [[[-1.5497]],

         [[ 3.8550]],

         [[-1.0453]]],


        [[[ 0.9808]],

         [[ 0.2446]],

         [[ 1.0353]]]], device='cuda:0')
torch.Size([3, 3, 1, 1])
cell0.Whi_mu
tensor([[[[-0.3148]],

         [[ 0.1252]],

         [[-0.5388]]],


        [[[-0.4797]],

         [[-0.1311]],

         [[-0.7403]]],


        [[[ 0.2932]],

         [[ 0.2068]],

         [[-0.6507]]]], device='cuda:0')
torch.Size([3, 3, 1, 1])
cell0.Wxf_mu
tensor([[[[ 1.5820]],

         [[-0.7307]],

         [[-1.0373]]],


        [[[-0.6130]],

         [[ 2.2079]],

         [[-1.3072]]],


        [[[ 0.0304]],

         [[-0.3185]],

         [[ 0.7252]]]], device='cuda:0')
torch.Size([3, 3, 1, 1])
cell0.Whf_mu
tensor([[[[ 0.3601]],

         [[ 0.0858]],

         [[ 0.7245]]],


        [[[ 0.7615]],

         [[

In [7]:
    print ('*******************  extract the weight from BayesConvLSTM  *********************')
    weight_model_BayesConvLSTM = {}
    # the weights are pytorch tensors
    for name, param in model_bayes.named_parameters():
        weight_model_BayesConvLSTM['{}'.format(name)] = param.data
        
    print ('*******************  check the weight from ConvLSTM  *********************')
    for i in weight_model_BayesConvLSTM:
        print('{}'.format(i))
        print('{}'.format(weight_model_BayesConvLSTM['{}'.format(i)]))
        print ("=========================")    

*******************  extract the weight from BayesConvLSTM  *********************
*******************  check the weight from ConvLSTM  *********************
cell0.Wxi_mu
tensor([[[[ 1.8897]],

         [[ 0.9244]],

         [[-1.5973]]],


        [[[-1.5497]],

         [[ 3.8550]],

         [[-1.0453]]],


        [[[ 0.9808]],

         [[ 0.2446]],

         [[ 1.0353]]]], device='cuda:0')
cell0.Whi_mu
tensor([[[[-0.3148]],

         [[ 0.1252]],

         [[-0.5388]]],


        [[[-0.4797]],

         [[-0.1311]],

         [[-0.7403]]],


        [[[ 0.2932]],

         [[ 0.2068]],

         [[-0.6507]]]], device='cuda:0')
cell0.Wxf_mu
tensor([[[[ 1.5820]],

         [[-0.7307]],

         [[-1.0373]]],


        [[[-0.6130]],

         [[ 2.2079]],

         [[-1.3072]]],


        [[[ 0.0304]],

         [[-0.3185]],

         [[ 0.7252]]]], device='cuda:0')
cell0.Whf_mu
tensor([[[[ 0.3601]],

         [[ 0.0858]],

         [[ 0.7245]]],


        [[[ 0.7615]],

         [

In [8]:
    ################################################################################# 
    #########            sample the weight of BayesConvLSTM                  ########
    #################################################################################
    namelist_weight = ['Wxi', 'Whi', 'Wxf', 'Whf', 'Wxc', 'Whc', 'Wxo', 'Who']
    namelist_bias = ['Wxi', 'Wxf', 'Wxc', 'Wxo']
    cell_index = len(hidden_channels)
    num_samples = 20
    
    for i in range(num_samples):
        weight_model_init = {}
        for s in range(cell_index):
            for w in namelist_weight:
                # take the mean
                mu = weight_model_BayesConvLSTM['cell{}.{}_mu'.format(s, w)].data
                # take the log_variance
                log_var = weight_model_BayesConvLSTM['cell{}.{}_log_alpha'.format(s, w)].data
                # nn.Tensor.new() Constructs a new tensor of the same data type as self tensor.
                # nn.Tensor.normal_() generate gaussian distribution with (0, 1)
                W_epsilon = mu.data.new(mu.size()).normal_()
                #print(W_epsilon.data)
                # compute the weight for convlstm
                weight = mu + torch.sqrt(torch.exp(log_var) * mu * mu + 1e-10) * W_epsilon
                weight_model_init['cell{}.{}.weight'.format(s, w)] = weight.data
            for b in namelist_bias:
                weight_model_init['cell{}.{}.bias'.format(s, b)] = weight_model_BayesConvLSTM['cell{}.{}_bias'.format(s, b)].data
        # create ConvLSTM network
        model_convlstm = dlacs.ConvLSTM.ConvLSTM(input_channels, hidden_channels, kernel_size,
                                                 weight_dict = weight_model_init).to(device)
        # save the ConvLSTM sample
        torch.save(model_convlstm.state_dict(), os.path.join(output_path,'sample_convlstm_lorenz84_ex_{}.pkl'.format(i)))

In [9]:
    print ('*******************  check the weight calculated weight  *********************')
    for i in weight_model_init:
        print('{}'.format(i))
        print('{}'.format(weight_model_init['{}'.format(i)]))
        print ("=========================")

*******************  check the weight calculated weight  *********************
cell0.Wxi.weight
tensor([[[[ 2.2707]],

         [[-0.2108]],

         [[ 0.6488]]],


        [[[ 0.9039]],

         [[ 1.4220]],

         [[ 1.1625]]],


        [[[ 1.3825]],

         [[-0.7441]],

         [[ 1.1763]]]], device='cuda:0')
cell0.Whi.weight
tensor([[[[-0.2926]],

         [[-0.0830]],

         [[ 0.5503]]],


        [[[ 1.5657]],

         [[-0.5148]],

         [[ 0.9871]]],


        [[[ 1.2935]],

         [[-0.7567]],

         [[-0.4714]]]], device='cuda:0')
cell0.Wxf.weight
tensor([[[[ 2.0355]],

         [[-0.7379]],

         [[-1.3181]]],


        [[[-0.2857]],

         [[ 1.5733]],

         [[-0.9062]]],


        [[[-0.0604]],

         [[-0.6453]],

         [[ 1.3553]]]], device='cuda:0')
cell0.Whf.weight
tensor([[[[ 0.2230]],

         [[ 0.0509]],

         [[ 1.0283]]],


        [[[ 0.2621]],

         [[-0.0272]],

         [[-0.2198]]],


        [[[ 0.9808]],

 

In [10]:
    print ('*******************  check the weight from ConvLSTM  *********************')
    for name, param in model_convlstm.named_parameters():
        if param.requires_grad:
            print (name)
            print (param.data)
            print (param.size())
            print ("=========================")

*******************  check the weight from ConvLSTM  *********************
cell0.Wxi.weight
tensor([[[[ 2.2707]],

         [[-0.2108]],

         [[ 0.6488]]],


        [[[ 0.9039]],

         [[ 1.4220]],

         [[ 1.1625]]],


        [[[ 1.3825]],

         [[-0.7441]],

         [[ 1.1763]]]], device='cuda:0')
torch.Size([3, 3, 1, 1])
cell0.Wxi.bias
tensor([-0.7629, -0.1383,  0.7092], device='cuda:0')
torch.Size([3])
cell0.Whi.weight
tensor([[[[-0.2926]],

         [[-0.0830]],

         [[ 0.5503]]],


        [[[ 1.5657]],

         [[-0.5148]],

         [[ 0.9871]]],


        [[[ 1.2935]],

         [[-0.7567]],

         [[-0.4714]]]], device='cuda:0')
torch.Size([3, 3, 1, 1])
cell0.Wxf.weight
tensor([[[[ 2.0355]],

         [[-0.7379]],

         [[-1.3181]]],


        [[[-0.2857]],

         [[ 1.5733]],

         [[-0.9062]]],


        [[[-0.0604]],

         [[-0.6453]],

         [[ 1.3553]]]], device='cuda:0')
torch.Size([3, 3, 1, 1])
cell0.Wxf.bias
tensor([-1.16

In [11]:
    print ('*******************  reload saved ConvLSTM model  *********************')
    # load model parameters
    model_reload = dlacs.ConvLSTM.ConvLSTM(input_channels, hidden_channels, kernel_size).to(device)
    model_reload.load_state_dict(torch.load(os.path.join(output_path, 'sample_convlstm_lorenz84_ex_2.pkl'),
                                     map_location=device))
    # load entire model
    #model_reload = torch.load(os.path.join(output_path, 'sample_convlstm_lorenz84_0.pkl'))
    print(model_reload)

*******************  reload saved ConvLSTM model  *********************
ConvLSTM(
  (cell0): ConvLSTMCell(
    (Wxi): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
    (Whi): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (Wxf): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
    (Whf): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (Wxc): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
    (Whc): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (Wxo): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
    (Who): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1), bias=False)
  )
)


In [12]:
    print ('*******************  check the weight from reloaded ConvLSTM  *********************')
    for name, param in model_reload.named_parameters():
        if param.requires_grad:
            print (name)
            print (param.data)
            print (param.size())
            print ("=========================")

*******************  check the weight from reloaded ConvLSTM  *********************
cell0.Wxi.weight
tensor([[[[ 2.3011]],

         [[-0.1830]],

         [[ 0.6728]]],


        [[[ 0.8516]],

         [[ 1.3877]],

         [[ 1.2283]]],


        [[[ 1.4883]],

         [[-0.7614]],

         [[ 1.2280]]]], device='cuda:0')
torch.Size([3, 3, 1, 1])
cell0.Wxi.bias
tensor([-0.7629, -0.1383,  0.7092], device='cuda:0')
torch.Size([3])
cell0.Whi.weight
tensor([[[[-0.2412]],

         [[-0.0796]],

         [[ 0.8901]]],


        [[[ 1.4224]],

         [[-0.4063]],

         [[ 1.1881]]],


        [[[ 1.1842]],

         [[-0.6811]],

         [[-0.3798]]]], device='cuda:0')
torch.Size([3, 3, 1, 1])
cell0.Wxf.weight
tensor([[[[ 2.1109]],

         [[-0.7052]],

         [[-1.2984]]],


        [[[-0.3014]],

         [[ 1.5255]],

         [[-0.9114]]],


        [[[-0.0608]],

         [[-0.6852]],

         [[ 1.4109]]]], device='cuda:0')
torch.Size([3, 3, 1, 1])
cell0.Wxf.bias
tens